In [ ]:
# hide

from pprint import pprint

%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp persistence

<IPython.core.display.Javascript object>

In [ ]:
# export

import json
import sqlite3

from will_it_saturate.core import BaseRepository


class SqliteRepository(BaseRepository):
    connection: sqlite3.Connection

    class Config:
        arbitrary_types_allowed = True

    def execute_stmt(self, stmt, ignore_error=False):
        cursor = self.connection.cursor()
        try:
            cursor.execute(stmt)
        except sqlite3.OperationalError as exc:
            if not ignore_error:
                raise (exc)

    def create_benchmark_table(self):
        stmt = """
            create table benchmark (
                benchmark_id INTEGER PRIMARY KEY,
                created DATE DEFAULT (datetime('now','localtime')),
                machine_id TEXT NOT NULL UNIQUE,
                uname TEXT NOT NULL,
                data TEXT NOT NULL
            )
        """
        self.execute_stmt(stmt, ignore_error=True)

    def create_result_table(self):
        stmt = """
            create table result (
                result_id INTEGER PRIMARY KEY,
                benchmark_id integer not NULL,
                created DATE DEFAULT (datetime('now','localtime')),
                server text not NULL,
                client text not NULL,
                file_size integer not NULL,
                complete_size integer not NULL,
                elapsed real not NULL,
                FOREIGN KEY(benchmark_id) REFERENCES benchmark(benchmark_id)
            )
        """
        self.execute_stmt(stmt, ignore_error=True)

    def create_tables(self):
        self.create_benchmark_table()
        self.create_result_table()

    def add_benchmark(self, benchmark):
        stmt = """
            insert into benchmark (machine_id, uname, data)
            values (?, ?, ?)
        """
        cursor = self.connection.cursor()
        machine_id = benchmark.machine_id
        uname = benchmark.uname_json
        data = benchmark.json()
        try:
            cursor = cursor.execute(stmt, [machine_id, uname, data])
        except sqlite3.IntegrityError as e:
            pass
        self.connection.commit()

    def get_benchmark(self, benchmark):
        stmt = """
            select *
              from benchmark
             where machine_id=?
        """
        cursor = self.connection.cursor()
        cursor.execute(stmt, [benchmark.machine_id])
        return cursor.fetchone()

    def get_benchmark_id(self, benchmark):
        row = self.get_benchmark(benchmark)
        if row is None:
            return row
        return row[0]

    def get_result(self, benchmark, result):
        benchmark_id = self.get_benchmark_id(benchmark)
        if benchmark_id is None:
            self.add_benchmark(benchmark)
        benchmark_id = self.get_benchmark_id(benchmark)
        assert benchmark_id is not None
        stmt = """
            select *
              from result
             where benchmark_id=?
               and server=?
               and client=?
               and file_size=?
               and complete_size=?
        """
        cursor = self.connection.cursor()
        cursor.execute(
            stmt,
            [
                benchmark_id,
                result.server,
                result.client,
                result.file_size,
                result.complete_size,
            ],
        )
        row = cursor.fetchone()
        if row is None:
            return result
        result.elapsed = row[7]
        return result

    def add_result(self, benchmark, result):
        benchmark_id = self.get_benchmark_id(benchmark)
        columns = [
            "benchmark_id",
            "server",
            "client",
            "file_size",
            "complete_size",
            "elapsed",
        ]
        columns_str = ",".join(columns)
        values_str = ",".join(["?" for c in columns])
        stmt = f"""
            insert into result ({columns_str})
            values ({values_str})
        """
        cursor = self.connection.cursor()
        result_dict = result.dict()
        result_dict["benchmark_id"] = benchmark_id
        values = [result_dict[c] for c in columns]
        try:
            cursor = cursor.execute(stmt, values)
        except sqlite3.IntegrityError as e:
            pass
        self.connection.commit()

    @classmethod
    def build_repository(cls, database_name):
        conn = sqlite3.connect(database_name)
        repo = cls(connection=conn)
        repo.create_tables()
        return repo

<IPython.core.display.Javascript object>

# Usage

In [ ]:
conn = sqlite3.connect("example.db")
repo = SqliteRepository(connection=conn)
repo.create_tables()

<IPython.core.display.Javascript object>

In [ ]:
from will_it_saturate.core import Benchmark
from will_it_saturate.core import BenchmarkResult

benchmark = Benchmark()
benchmark = Benchmark()
repo.add_benchmark(benchmark)
repo.add_benchmark(benchmark)  # should not raise integrity exception

<IPython.core.display.Javascript object>

In [ ]:
repo.get_benchmark(benchmark)

(1,
 '2021-02-28 16:27:43',
 'C02ZPB8KMD6T',
 '["Darwin", "ephesmpb.fritz.box", "20.3.0", "Darwin Kernel Version 20.3.0: Thu Jan 21 00:07:06 PST 2021; root:xnu-7195.81.3~1/RELEASE_X86_64", "x86_64", "i386"]',
 '{"duration": 30, "bandwidth": 125000000, "cpuinfo": {"python_version": "3.9.2.final.0 (64 bit)", "cpuinfo_version": [7, 0, 0], "cpuinfo_version_string": "7.0.0", "arch": "X86_64", "bits": 64, "count": 16, "arch_string_raw": "x86_64", "vendor_id_raw": "GenuineIntel", "brand_raw": "Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz", "hz_advertised_friendly": "2.3000 GHz", "hz_actual_friendly": "2.3000 GHz", "hz_advertised": [2300000000, 0], "hz_actual": [2300000000, 0], "l2_cache_size": 262144, "stepping": 13, "model": 158, "family": 6, "flags": ["1gbpage", "3dnowprefetch", "abm", "acapmsr", "acpi", "adx", "aes", "apic", "avx", "avx1.0", "avx2", "bmi1", "bmi2", "clflush", "clflushopt", "clfsh", "clfsopt", "cmov", "cx16", "cx8", "de", "ds", "ds_cpl", "dscpl", "dtes64", "dts", "em64t", "erms

<IPython.core.display.Javascript object>

In [ ]:
benchmark.json()

'{"duration": 30, "bandwidth": 125000000, "cpuinfo": {"python_version": "3.9.2.final.0 (64 bit)", "cpuinfo_version": [7, 0, 0], "cpuinfo_version_string": "7.0.0", "arch": "X86_64", "bits": 64, "count": 16, "arch_string_raw": "x86_64", "vendor_id_raw": "GenuineIntel", "brand_raw": "Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz", "hz_advertised_friendly": "2.3000 GHz", "hz_actual_friendly": "2.3000 GHz", "hz_advertised": [2300000000, 0], "hz_actual": [2300000000, 0], "l2_cache_size": 262144, "stepping": 13, "model": 158, "family": 6, "flags": ["1gbpage", "3dnowprefetch", "abm", "acapmsr", "acpi", "adx", "aes", "apic", "avx", "avx1.0", "avx2", "bmi1", "bmi2", "clflush", "clflushopt", "clfsh", "clfsopt", "cmov", "cx16", "cx8", "de", "ds", "ds_cpl", "dscpl", "dtes64", "dts", "em64t", "erms", "est", "f16c", "fma", "fpu", "fpu_csds", "fxsr", "ht", "htt", "ibrs", "intel_pt", "invpcid", "ipt", "l1df", "lahf", "lahf_lm", "lzcnt", "mca", "mce", "mdclear", "mmx", "mon", "monitor", "movbe", "mpx", "msr"

<IPython.core.display.Javascript object>

In [ ]:
# repo.get_benchmark(benchmark)

<IPython.core.display.Javascript object>

In [ ]:
benchmark.machine_id = "foobar"

<IPython.core.display.Javascript object>

In [ ]:
file_size = 10 ** 6
complete_size = 100 * file_size
result = BenchmarkResult(
    server="nginx",
    client="httpx",
    file_size=file_size,
    complete_size=complete_size,
)

result = repo.get_result(benchmark, result)
print(result.elapsed)

3.0


<IPython.core.display.Javascript object>

In [ ]:
from will_it_saturate.core import BenchmarkResult

file_size = 10 ** 6
complete_size = 100 * file_size
result = BenchmarkResult(
    server="nginx",
    client="httpx",
    file_size=file_size,
    elapsed=3.0,
    complete_size=complete_size,
)
print(result.dict())

{'server': 'nginx', 'client': 'httpx', 'file_size': 1000000, 'elapsed': 3.0, 'complete_size': 100000000}


<IPython.core.display.Javascript object>

In [ ]:
repo.add_result(benchmark, result)


            insert into result (benchmark_id,server,client,file_size,complete_size,elapsed)
            values (?,?,?,?,?,?)
        


<IPython.core.display.Javascript object>

# Export

In [ ]:
# dont_test

from nbdev.export import notebook2script

notebook2script()

Converted 00_core.ipynb.
Converted 01_serve_files_uvicorn.ipynb.
Converted 02_fastapi_uvicorn_server.ipynb.
Converted 03_nginx_docker_server.ipynb.
Converted 04_benchmark_clients.ipynb.
Converted 05_persistence.ipynb.
Converted 06_run_benchmark.ipynb.
Converted index.ipynb.
Converted run_wrk_benchmark_test.ipynb.
Converted use_nginx_to_serve_files_test.ipynb.


<IPython.core.display.Javascript object>